In [1]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools
import os
from time import time
from pathlib import Path
import os

In [2]:
def create_urls_table_blank(step = 10, overwrite=True,file_name="urls_table.csv"):
    '''
    Puts together all the urls that would list (by accessing every one of those) all the data of the web site.
    step = how many rows has the table displayed per url.
    
    For instance if you take step = 200 the pages will show 200 companies per page.

    '''
    
    # get the last number added
    total = 271605 # according to the web the total amount of entries is this one
    total_urls = total//step
    parameters = [i*step for i in range(total_urls)] + [total]
    all_urls =[]
    for i,para in enumerate(parameters):
        url = f"https://www.proplanta.de/Agrarsubventionen-2021-Liste-der-Empfaenger/proplanta_karten.php?ROalAk=271605&LaZ={step}&LsZ={para}&ROalAk=271605&SELECTID=1653383570&SEARCH_SHOWBEGS=1653383570"
        all_urls.append(url)
    
    df_urls = pd.DataFrame()
    
    df_urls['url'] = all_urls
    df_urls = df_urls[[col for col in df_urls.columns if 'Unn' not in col]]
    df_urls['DONE'] = 'NO'
    
    if overwrite:
        #create_urls_table_blank
        df_urls.to_csv(file_name)
        rows = len(df_urls)
        print(f"{file_name} created with {rows} rows, with every row being a table of {step} companies.")
    else:
        raise f"There is already a file named {file_name}: delete it manually and run this again"
    
    return df_urls


In [3]:
def get_remaining_urls(step=10,file_name="urls_table.csv"):
    '''
    returns a list with all the urls still to do
    '''
    if os.path.exists(file_name) == False:
        create_urls_table_blank(step)
        
    df = pd.read_csv(file_name)
    rows = len(df)
    
    mask = df['DONE'] =='NO'
    #only gives back the data not mark as DONE YES
    result = df.loc[mask,'url'].tolist()
    print(f"{file_name} read with {rows} rows with {len(result)} remaining to be dowloaded.")
    return result


In [4]:
def mark_one_url_as_done(query_value_for_column, 
                         col_input='url',
                         col_to_change = 'DONE',
                         value_for_change = 'YES',
                         file_name="urls_table.csv"):
    '''
    since we dont want to make all the 270000 calls in one go this is a way to mark up bit a bit what was done
    '''
    df = pd.read_csv(file_name)
    if col_input not in df.columns:
        raise f"wooow pass a col name that is present in the dataframe please. {col_input} is not in {df.columns}"
    # read file and modify it
    
    mask1 = df[col_input] == query_value_for_column
    # caveats Try using .loc[row_indexer,col_indexer] = value instead
    df.loc[mask1,col_to_change]=value_for_change
    df.to_csv(file_name, index=False)
    return # nothing is returned because the table is saved again

In [5]:
def get_data_of_one_url_table(url, DEBUG=False):
    '''
    Make a GET request to fetch the raw HTML content of the url
    INPUT:
        url: like: 'https://www.proplanta.de/Agrarsubventionen-2021-Liste-der-Empfaenger/proplanta_karten.php?ROalAk=271605&LaZ=2&LsZ=2&ROalAk=271605&SELECTID=1653383570&SEARCH_SHOWBEGS=1653383570'
    RETURNS:
        list of dicts in the form of:
        [{'total_sum': '12.649.000,18',
          'link_total_sum': '',
          'company': 'Land Mecklenburg-Vorpommern Ministerium für – 19061 Schwerin, Landeshauptstadt',
          'link_company': '/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html',
          'location': 'Kreisfreie Stadt Schwerin, Landeshauptstadt',
          'link_location': '/karten/kreisfreie_stadt_schwerin,_landeshauptstadt-agrarsubventionen_2021-landkreis_13004.html',
          'Lander': 'Mecklenburg-Vorpommern',
          'link_lander': '/karten/mecklenburg-vorpommern-agrarsubventionen_2021-bundesland_5a61ca66ef23199eb6afa5e9bb4779f1.html'},
          {...
          ]
    
    This lists of dicts corresponds to what you see when accesing one single page
    
    
    '''
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    
    table = soup.find("table",{"class":"display dataTable"})
    if DEBUG: print('>>>>>>>>>>>>>>>>>>>>>>',url, table)
    
    all_data=[]
    for i,row in enumerate(table.find_all('tr')):
        if row.td is not None:
            row_data_per_company = [(td.text,td.a.get('href')) if td.a and td.a.get('href') else (td.text,'') for td in row.find_all('td')]
            elements_of_one_row=[]
            
            for tupple in row_data_per_company:
                elements_of_one_row.extend(tupple)
            if DEBUG: print(elements_of_one_row)
            keys = ['Foerdersumme', 'link_total_sum','Begünstigter','link_company','Landkreis','link_location','BundesLand','link_lander']
            d = dict(zip(keys,elements_of_one_row))
            
            all_data.append(d)
    
    return all_data

In [6]:
# example:
DEBUG=False
if DEBUG:
    url='https://www.proplanta.de/Agrarsubventionen-2021-Liste-der-Empfaenger/proplanta_karten.php?ROalAk=271605&LaZ=2&LsZ=2&ROalAk=271605&SELECTID=1653383570&SEARCH_SHOWBEGS=1653383570'
    print(url)
    get_data_of_one_url_table(url)

In [7]:
def get_basis_praemie_from_url(company_link,DEBUG=False):
    if DEBUG: print("in soup:",company_link)
    html_content = requests.get(company_link).text
    soup = BeautifulSoup(html_content, "lxml")
    try:
        t3 = soup.find(lambda tag:tag.name=="h3" and "Basisprämie" in tag.text).find_next_sibling().find_next_sibling("p").find("span").contents
    except:
        t3 = ''
    basis_praemie = t3
    if type(basis_praemie)==type(["unalista"]):
        basis_praemie = basis_praemie[0]
    
    return basis_praemie

In [8]:
DEBUG=False
if DEBUG:
    # no baispramie
    company_link1 = "https://proplanta.de/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html"
    # with basis pramie
    company_link2 = "https://www.proplanta.de/karten/agrarsubventionen_2021-empfaengerb23c81b03f98afededf2dce2fbb239ea.html"
    company_link= company_link1
    print(company_link)
    print(company_link)
    html_content = requests.get(company_link).text
    soup = BeautifulSoup(html_content, "lxml")
    print(get_basis_praemie_from_url(company_link))
    print(soup)
    

In [9]:
def add_basispramie_to_one_company(list_of_company_dicts, DEBUG=False):
    '''
    INPUT:
        [{'total_sum': '12.649.000,18',
          'link_total_sum': '',
          'company': 'Land Mecklenburg-Vorpommern Ministerium für – 19061 Schwerin, Landeshauptstadt',
          'link_company': '/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html',
          'location': 'Kreisfreie Stadt Schwerin, Landeshauptstadt',
          'link_location': '/karten/kreisfreie_stadt_schwerin,_landeshauptstadt-agrarsubventionen_2021-landkreis_13004.html',
          'Lander': 'Mecklenburg-Vorpommern',
          'link_lander': '/karten/mecklenburg-vorpommern-agrarsubventionen_2021-bundesland_5a61ca66ef23199eb6afa5e9bb4779f1.html'},
  ]
  OUTPUT:
      {'total_sum': '12.649.000,18',
      'link_total_sum': '',
      'company': 'Land Mecklenburg-Vorpommern Ministerium für – 19061 Schwerin, Landeshauptstadt',
      'link_company': '/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html',
      'location': 'Kreisfreie Stadt Schwerin, Landeshauptstadt',
      'link_location': '/karten/kreisfreie_stadt_schwerin,_landeshauptstadt-agrarsubventionen_2021-landkreis_13004.html',
      'Lander': 'Mecklenburg-Vorpommern',
      'link_lander': '/karten/mecklenburg-vorpommern-agrarsubventionen_2021-bundesland_5a61ca66ef23199eb6afa5e9bb4779f1.html'},
      'basis_praemie': NEW DATA HERE
  
  Explanation:
  Out of the dict corresponding to one company the url is built up and the data is fetched with requeswt and parsed with soup
  
    '''
    new_list_of_company_dicts = []
    for company_dict in list_of_company_dicts:
        basis_link = 'https://proplanta.de'
        company_link = basis_link + company_dict['link_company']
        company_dict['company_url'] = company_link
        company_dict['basis_praemie'] = get_basis_praemie_from_url(company_link, DEBUG)
        new_list_of_company_dicts.append(company_dict)
    return new_list_of_company_dicts

In [10]:
DEBUG=False
if DEBUG:
    example="https://www.proplanta.de/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html"
    example2 = [{'total_sum': '12.649.000,18',
          'link_total_sum': '',
          'company': 'Land Mecklenburg-Vorpommern Ministerium für – 19061 Schwerin, Landeshauptstadt',
          'link_company': '/karten/agrarsubventionen_2021-empfaenger22becefcfe9631bdec63c6f251515d00.html',
          'location': 'Kreisfreie Stadt Schwerin, Landeshauptstadt',
          'link_location': '/karten/kreisfreie_stadt_schwerin,_landeshauptstadt-agrarsubventionen_2021-landkreis_13004.html',
          'Lander': 'Mecklenburg-Vorpommern',
          'link_lander': '/karten/mecklenburg-vorpommern-agrarsubventionen_2021-bundesland_5a61ca66ef23199eb6afa5e9bb4779f1.html'},
  ]
    print(add_basispramie_to_one_company(example2, DEBUG=True))
    

In [19]:
def add_dict_to_df(file_name_for_output_data,
                   dict_to_add,
                   file_name = '',
                   DEBUG=False):
    
    df_to_add=pd.DataFrame(dict_to_add)
        
    if os.path.exists(file_name_for_output_data):
        df= pd.read_csv(file_name_for_output_data)
        
        # add the new data
        out = pd.concat([df_to_add, df])
        if DEBUG:
            display(df)
            display(df_to_add)
    else:
        out=df_to_add
    rows = len(out)
    print(f"total {rows} companies accumulated")
    out.to_csv(file_name_for_output_data, index=False) 

# main code

In [20]:
# The following are the only two parameters to modify and then run the next cell
# mumber of rows per table in every call to the website
def run_process(lines_per_table = 7,
                nr_of_tables_to_download_today = 5,
                first_time_running_the_process = False,
                file_name_for_output_data = "companies.csv",
                DEBUG=False):
    tic = time()
    if first_time_running_the_process:
        create_urls_table_blank(step=lines_per_table)
        if os.path.exists(file_name_for_output_data):
            os.remove(file_name_for_output_data)

    all_urls_for_all_data_still_to_do = get_remaining_urls(file_name="urls_table.csv")   

    # all the urls with the parameters decided at the beginning, how many per page.
    if len(all_urls_for_all_data_still_to_do)>nr_of_tables_to_download_today:
        all_urls_for_all_data_still_to_do = all_urls_for_all_data_still_to_do[:nr_of_tables_to_download_today]
    else:
        all_urls_for_all_data_still_to_do = all_urls_for_all_data_still_to_do


    for i, url_table in enumerate(all_urls_for_all_data_still_to_do):
        # table of companies
        data_for_one_table = get_data_of_one_url_table(url_table)
        #print(len(data_for_one_table))
        #print(data_for_one_table)
        data_for_one_table_with_basis_premie = add_basispramie_to_one_company(data_for_one_table)
        #print(data_for_one_table_with_basis_premie)
        # mark this url as done:
        mark_one_url_as_done(url_table,
                             col_input='url',
                             col_to_change = 'DONE',
                             value_for_change = 'YES',
                             file_name="urls_table.csv")


        add_dict_to_df(file_name_for_output_data,
                       data_for_one_table_with_basis_premie)
        
        print(i, ' of ', len(all_urls_for_all_data_still_to_do), ' >>>',url_table)
    toc=time()
    nice_time = "{:.2f}".format(toc-tic)
    print(f"downloaded {lines_per_table*nr_of_tables_to_download_today} in {nice_time} seconds.")

In [ ]:
lines_per_table = 20 # how many lines per table
nr_of_tables_to_download_today = 5 # how many tables to download
first_time_running_the_process = True # make it true and all starts from scratch. But the make it False if you continue the next day
file_name_for_output_data = "companies.csv"
DEBUG = False #if true prints info per table downloaded.
run_process(lines_per_table ,
                nr_of_tables_to_download_today,
                first_time_running_the_process,
                file_name_for_output_data,
                DEBUG)
    

urls_table.csv created with 13581 rows, with every row being a table of 20 companies.
urls_table.csv read with 13581 rows with 13581 remaining to be dowloaded.
